- Spotify CSV File: data.csv
- Spotify API: https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-audio-features
* .gitignore file contains config.py file to store the spotify API client username and password. 

In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine

SPOTIFY CSV FILE - Extract and Transform



Extract CSV into DataFrames

In [2]:
spotify_csv = "resources/data.csv"

Transform CSV DataFrame

In [3]:
csv_df = pd.read_csv(spotify_csv)
csv_df.head()
# Keep columns: id, name, artist, key, tempo, year, popularity

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [4]:
# Select only the columns required
new_csv_df = csv_df[['id','name', 'artists','key','tempo','year','popularity']].copy()
new_csv_df.head()

# Drop ids with duplicates to ensure no double up
new_csv_df = new_csv_df.drop_duplicates(subset=['id'])

# TEST: new_csv_df.info()
# There are 174389 row entries before drop duplicates
# There are 172230 row entries after drop duplicates

# Rename columns
new_csv_df=new_csv_df.rename(columns={'name':'s_name', 'artists': 's_artist','key':'s_key','tempo':'s_tempo','year':'s_year','popularity':'s_popularity'})
new_csv_df


,id,s_name,s_artist,s_key,s_tempo,s_year,s_popularity
0,0cS0A1fUEUd1EW3FcF8AEI,Keep A Song In Your Soul,['Mamie Smith'],5,149.976,1920,12
1,0hbkKFIJm7Z05H8Zl9w30f,I Put A Spell On You,"[""Screamin' Jay Hawkins""]",5,86.889,1920,7
2,11m7laMUgmOKqI3oYzuhne,Golfing Papa,['Mamie Smith'],0,97.600,1920,4
3,19Lc5SfJJ5O1oaxY0fpwfh,True House Music - Xavier Santos & Carlos Gomi...,['Oscar Velazquez'],2,127.997,1920,17
4,2hJjbsLCytGsnAHfdsLejp,Xuniverxe,['Mixe'],10,122.076,1920,2
...,...,...,...,...,...,...,...
174379,45XnLMuqf3vRfskEAMUeCH,A Little More,['Alessia Cara'],4,94.710,2021,0
174381,4pPFI9jsguIh3wC7Otoyy8,A Little More,['Alessia Cara'],4,94.710,2021,0
174383,52YtxLVUyvtiGPxwwxayHZ,A Little More,['Alessia Cara'],4,94.710,2021,0
174385,7tue2Wemjd0FZzRtDrQFZd,A Little More,['Alessia Cara'],4,94.710,2021,0


SPOTIFY API

- Objective: To extract the time signature of all tracks on spotify

In [5]:
#Dependences
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from config import *
import requests
# from config import cid
# from config import secret

In [6]:
CLIENT_ID = cid
CLIENT_SECRET = secret

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [7]:
id_list= []
for index, row in csv_df.iterrows():
    id_list.append(row['id'])
    # print(row['id'])
    # print(id_list)
print('Completed populating id_list.')

Completed populating id_list.


In [13]:
id_timesig= []
count=0

# url = "https://api.spotify.com/v1/audio-features/{id}" 
BASE_URL = 'https://api.spotify.com/v1/'

# for item in id_list[:5]:
for item in id_list:
    # Track ID from the URI
    # track_id = '6y0igZArWVi6Iz0rj35c1Y'
    track_id = item
    # actual GET request with proper header
    r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    r = r.json()
    # print(r)

    try:
        # print(f"id: {r['id']}")
        # print(f"time_signature: {r['time_signature']}")
        # print('-------------------------')
        new_song = r['id'],r['time_signature']
        id_timesig.append(new_song)
        count+=1
        # print(count)
        if (count == 5000):
            print('Achieved 5000 entries')
        elif (count == 20000):
            print('Achieved 20000 entries')
        elif (count == 50000):
            print('Achieved 50000 entries')
        elif (count == 70000):
            print('Achieved 70000 entries')
        elif (count == 100000):
            print('Achieved 100000 entries')
        elif (count == 150000):
            print('Achieved 150000 entries')

    # except ValueError:
    except ValueError or KeyError:
        print("JSON Decode Error on id or time signature")

print("All ids and time signatures appended")

KeyError: 'id'

In [23]:
id_timesig_df = pd.DataFrame(id_timesig)
id_timesig_df

,0,1
0,0cS0A1fUEUd1EW3FcF8AEI,4
1,0hbkKFIJm7Z05H8Zl9w30f,4
2,11m7laMUgmOKqI3oYzuhne,4
3,19Lc5SfJJ5O1oaxY0fpwfh,4
4,2hJjbsLCytGsnAHfdsLejp,4
...,...,...
5817,5jSpy3KyBwLxruntfWbcDR,4
5818,5ttXtQa7Ru0AlS9ox0v0kC,4
5819,6cD744rp35E6G1XfVkoKj6,4
5820,0KKOffDuUhWFxyrA4yCbom,4


In [36]:
id_timesig_df=id_timesig_df.rename(columns={0: "id", 1: "s_timesig"})
#Save to csv file
id_timesig_df.to_csv('resources/id_timesig_df.csv', index=False)
id_timesig_df


,id,s_timesig
0,0cS0A1fUEUd1EW3FcF8AEI,4
1,0hbkKFIJm7Z05H8Zl9w30f,4
2,11m7laMUgmOKqI3oYzuhne,4
3,19Lc5SfJJ5O1oaxY0fpwfh,4
4,2hJjbsLCytGsnAHfdsLejp,4
...,...,...
5817,5jSpy3KyBwLxruntfWbcDR,4
5818,5ttXtQa7Ru0AlS9ox0v0kC,4
5819,6cD744rp35E6G1XfVkoKj6,4
5820,0KKOffDuUhWFxyrA4yCbom,4


Load

In [18]:
# Connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/music_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [19]:
# Create a music_db. Add the tables into postgres database as per schema.sql
engine.table_names()

['spotify_api', 'spotify_csv']

In [17]:
# Load pandas dataframe new_csv_df to database music_db, table spotify_csv
new_csv_df.to_sql(name='spotify_csv', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "spotify_csv_pkey"
DETAIL:  Key (id)=(0cS0A1fUEUd1EW3FcF8AEI) already exists.

[SQL: INSERT INTO spotify_csv (id, s_name, s_artist, s_key, s_tempo, s_year, s_popularity) VALUES (%(id)s, %(s_name)s, %(s_artist)s, %(s_key)s, %(s_tempo)s, %(s_year)s, %(s_popularity)s)]
[parameters: ({'id': '0cS0A1fUEUd1EW3FcF8AEI', 's_name': 'Keep A Song In Your Soul', 's_artist': "['Mamie Smith']", 's_key': 5, 's_tempo': 149.976, 's_year': 1920, 's_popularity': 12}, {'id': '0hbkKFIJm7Z05H8Zl9w30f', 's_name': 'I Put A Spell On You', 's_artist': '["Screamin\' Jay Hawkins"]', 's_key': 5, 's_tempo': 86.889, 's_year': 1920, 's_popularity': 7}, {'id': '11m7laMUgmOKqI3oYzuhne', 's_name': 'Golfing Papa', 's_artist': "['Mamie Smith']", 's_key': 0, 's_tempo': 97.6, 's_year': 1920, 's_popularity': 4}, {'id': '19Lc5SfJJ5O1oaxY0fpwfh', 's_name': 'True House Music - Xavier Santos & Carlos Gomix Remix', 's_artist': "['Oscar

In [ ]:
# Load pandas dataframe id_timesig_df to database music_db, table spotify_api
id_timesig_df.to_sql(name='spotify_api', con=engine, if_exists='append', index=False)

# Alternatively, id_timesig_df pandas dataframe has been saved as a csv file. This csv file can be loaded into postgres database directly in table 'spotify_api'
# Relative csv link location: resources/id_timesig_df.csv


In [21]:
ts_df = pd.read_csv('resources/id_timesig_df.csv')
ts_df.head()

,Unnamed: 0,id,s_timesig
0,0,0cS0A1fUEUd1EW3FcF8AEI,4
1,1,0hbkKFIJm7Z05H8Zl9w30f,4
2,2,11m7laMUgmOKqI3oYzuhne,4
3,3,19Lc5SfJJ5O1oaxY0fpwfh,4
4,4,2hJjbsLCytGsnAHfdsLejp,4


In [27]:
ts_df=ts_df[['id','s_timesig']]
ts_df

ts_df.to_csv('resources/id_timesig_df.csv', index=False) 